In [1]:
import os
import sys
import shutil
import numpy as np
sys.path.insert(0, '../scripts')

from eeg_cnn_lib import transfer_all_class, prepare_data_labels
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras import optimizers

# The images are in a folder named 'shapes/training'
train_dir_name = '../../../data/classification/valence/train'
validation_dir_name = '../../../data/classification/valence/validation'
test_dir_name = '../../../data/classification/valence/test'


# All images are 32 pixels
img_size = (32,32)
img_size_rgba = (32,32,4)

# The folder contains a subfolder for each class of shape
classes = sorted(os.listdir(train_dir_name))

Using TensorFlow backend.


In [2]:
import sys
import keras
print('Keras version:',keras.__version__)

from keras import backend as K

Keras version: 2.3.1


In [3]:
transfer_all_class(test_dir_name,train_dir_name,1.0,classes)
transfer_all_class(validation_dir_name,train_dir_name,1.0,classes)
transfer_all_class(train_dir_name,test_dir_name,0.1,classes)

In [4]:
X, Y = prepare_data_labels(classes, train_dir_name)
X=np.asarray(X)
Y=np.asarray(Y)

In [6]:
model = Sequential()

model.add(Conv2D(32, (3, 3), strides=1, padding='same', input_shape=img_size_rgba, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Conv2D(64, (3, 3), strides=1, padding='same', input_shape=img_size_rgba, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Conv2D(128, (3, 3), strides=1, padding='same', input_shape=img_size_rgba, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Flatten())

model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))

model.add(Dense(len(classes), activation='softmax'))

opt = optimizers.Adam(lr=0.001)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        1184      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)             

In [7]:
kf = KFold(n_splits=10, shuffle=False)
accuracies = []

In [8]:
for train_index, val_index in kf.split(X, Y):
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    transfer_all_class(validation_dir_name,train_dir_name,1.0,classes)
    for index in range(len(X_val)):
        class_label=''
        if(Y_val[index]==0):
            class_label=classes[0]
        else:
            class_label=classes[1]  
        #Then, copy the validation images to the validation folder
        shutil.move(train_dir_name+"/"+class_label+'/'+X_val[index], 
                    validation_dir_name+"/"+class_label+'/'+X_val[index])
        
    batch_size = 256
    epoch=100
    datagen = ImageDataGenerator(rescale=1./255)

    train_generator = datagen.flow_from_directory(
        train_dir_name,
        target_size=img_size,
        color_mode='rgba',
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = datagen.flow_from_directory(
        validation_dir_name,
        target_size=img_size,
        color_mode='rgba',
        batch_size=batch_size,
        class_mode=None)
        
    history=model.fit_generator(train_generator, 
                        epochs=epoch)
        
    predictions = model.predict_generator(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    y_true = validation_generator.classes
    accuracy=accuracy_score(y_true, y_predictions)
    accuracies.append(accuracy)
    print(accuracy)

Found 13772 images belonging to 2 classes.
Found 1531 images belonging to 2 classes.
Epoch 1/100
54/54 [==============================] - 8s 151ms/step - loss: 0.6870 - accuracy: 0.5677
Epoch 2/100
54/54 [==============================] - 9s 161ms/step - loss: 0.6806 - accuracy: 0.5814
Epoch 3/100
54/54 [==============================] - 8s 157ms/step - loss: 0.6796 - accuracy: 0.5814
Epoch 4/100
54/54 [==============================] - 8s 155ms/step - loss: 0.6794 - accuracy: 0.5814
Epoch 5/100
54/54 [==============================] - 8s 149ms/step - loss: 0.6785 - accuracy: 0.5814
Epoch 6/100
54/54 [==============================] - 9s 164ms/step - loss: 0.6770 - accuracy: 0.5813
Epoch 7/100
54/54 [==============================] - 9s 166ms/step - loss: 0.6735 - accuracy: 0.5818
Epoch 8/100
54/54 [==============================] - 9s 173ms/step - loss: 0.6721 - accuracy: 0.5840
Epoch 9/100
54/54 [==============================] - 10s 184ms/step - loss: 0.6694 - accuracy: 0.5881
Epoch

54/54 [==============================] - 12s 219ms/step - loss: 0.2255 - accuracy: 0.8962
Epoch 80/100
54/54 [==============================] - 10s 194ms/step - loss: 0.2235 - accuracy: 0.8997
Epoch 81/100
54/54 [==============================] - 11s 201ms/step - loss: 0.2148 - accuracy: 0.9002
Epoch 82/100
54/54 [==============================] - 11s 202ms/step - loss: 0.2208 - accuracy: 0.8973
Epoch 83/100
54/54 [==============================] - 11s 197ms/step - loss: 0.2164 - accuracy: 0.9007
Epoch 84/100
54/54 [==============================] - 11s 206ms/step - loss: 0.1993 - accuracy: 0.9109
Epoch 85/100
54/54 [==============================] - 11s 212ms/step - loss: 0.2110 - accuracy: 0.9018
Epoch 86/100
54/54 [==============================] - 11s 198ms/step - loss: 0.2090 - accuracy: 0.9018
Epoch 87/100
54/54 [==============================] - 11s 209ms/step - loss: 0.2048 - accuracy: 0.9060
Epoch 88/100
54/54 [==============================] - 12s 228ms/step - loss: 0.2040 - 

54/54 [==============================] - 12s 214ms/step - loss: 0.1221 - accuracy: 0.9464
Epoch 58/100
54/54 [==============================] - 14s 252ms/step - loss: 0.1248 - accuracy: 0.9455
Epoch 59/100
54/54 [==============================] - 12s 215ms/step - loss: 0.1194 - accuracy: 0.9484
Epoch 60/100
54/54 [==============================] - 14s 251ms/step - loss: 0.1195 - accuracy: 0.9475
Epoch 61/100
54/54 [==============================] - 11s 211ms/step - loss: 0.1222 - accuracy: 0.9474
Epoch 62/100
54/54 [==============================] - 12s 225ms/step - loss: 0.1295 - accuracy: 0.9442
Epoch 63/100
54/54 [==============================] - 12s 220ms/step - loss: 0.1296 - accuracy: 0.9439
Epoch 64/100
54/54 [==============================] - 11s 209ms/step - loss: 0.1272 - accuracy: 0.9462
Epoch 65/100
54/54 [==============================] - 11s 205ms/step - loss: 0.1197 - accuracy: 0.9473
Epoch 66/100
54/54 [==============================] - 12s 230ms/step - loss: 0.1197 - 

54/54 [==============================] - 12s 220ms/step - loss: 0.0966 - accuracy: 0.9588
Epoch 36/100
54/54 [==============================] - 11s 210ms/step - loss: 0.0992 - accuracy: 0.9575
Epoch 37/100
54/54 [==============================] - 12s 224ms/step - loss: 0.0982 - accuracy: 0.9587
Epoch 38/100
54/54 [==============================] - 12s 216ms/step - loss: 0.0987 - accuracy: 0.9588
Epoch 39/100
54/54 [==============================] - 11s 206ms/step - loss: 0.0945 - accuracy: 0.9609
Epoch 40/100
54/54 [==============================] - 11s 204ms/step - loss: 0.1003 - accuracy: 0.9573
Epoch 41/100
54/54 [==============================] - 12s 230ms/step - loss: 0.1004 - accuracy: 0.9597
Epoch 42/100
54/54 [==============================] - 11s 205ms/step - loss: 0.1032 - accuracy: 0.9580
Epoch 43/100
54/54 [==============================] - 11s 207ms/step - loss: 0.0949 - accuracy: 0.9601
Epoch 44/100
54/54 [==============================] - 12s 220ms/step - loss: 0.0939 - 

54/54 [==============================] - 11s 212ms/step - loss: 0.0836 - accuracy: 0.9659
Epoch 14/100
54/54 [==============================] - 11s 206ms/step - loss: 0.0943 - accuracy: 0.9612
Epoch 15/100
54/54 [==============================] - 11s 196ms/step - loss: 0.0866 - accuracy: 0.9646
Epoch 16/100
54/54 [==============================] - 11s 197ms/step - loss: 0.0962 - accuracy: 0.9591
Epoch 17/100
54/54 [==============================] - 11s 196ms/step - loss: 0.0849 - accuracy: 0.9647
Epoch 18/100
54/54 [==============================] - 11s 202ms/step - loss: 0.0794 - accuracy: 0.9676
Epoch 19/100
54/54 [==============================] - 12s 222ms/step - loss: 0.0843 - accuracy: 0.9670
Epoch 20/100
54/54 [==============================] - 11s 209ms/step - loss: 0.0844 - accuracy: 0.9644
Epoch 21/100
54/54 [==============================] - 11s 198ms/step - loss: 0.0759 - accuracy: 0.9681
Epoch 22/100
54/54 [==============================] - 11s 198ms/step - loss: 0.0831 - 

...

54/54 [==============================] - 12s 223ms/step - loss: 0.0488 - accuracy: 0.9817
Epoch 27/100
54/54 [==============================] - 14s 252ms/step - loss: 0.0603 - accuracy: 0.9784
Epoch 28/100
54/54 [==============================] - 12s 224ms/step - loss: 0.0522 - accuracy: 0.9814
Epoch 29/100
54/54 [==============================] - 14s 254ms/step - loss: 0.0520 - accuracy: 0.9803
Epoch 30/100
54/54 [==============================] - 12s 227ms/step - loss: 0.0533 - accuracy: 0.9781
Epoch 31/100
54/54 [==============================] - 13s 246ms/step - loss: 0.0463 - accuracy: 0.9836
Epoch 32/100
54/54 [==============================] - 13s 248ms/step - loss: 0.0496 - accuracy: 0.9799
Epoch 33/100
54/54 [==============================] - 12s 229ms/step - loss: 0.0531 - accuracy: 0.9798
Epoch 34/100
54/54 [==============================] - 14s 259ms/step - loss: 0.0551 - accuracy: 0.9798
Epoch 35/100
54/54 [==============================] - 12s 224ms/step - loss: 0.0459 - 

54/54 [==============================] - 12s 223ms/step - loss: 0.0569 - accuracy: 0.9781
Epoch 5/100
54/54 [==============================] - 11s 205ms/step - loss: 0.0518 - accuracy: 0.9812
Epoch 6/100
54/54 [==============================] - 11s 202ms/step - loss: 0.0590 - accuracy: 0.9776
Epoch 7/100
54/54 [==============================] - 11s 206ms/step - loss: 0.0498 - accuracy: 0.9823
Epoch 8/100
54/54 [==============================] - 14s 267ms/step - loss: 0.0489 - accuracy: 0.9813
Epoch 9/100
54/54 [==============================] - 14s 254ms/step - loss: 0.0500 - accuracy: 0.9810
Epoch 10/100
54/54 [==============================] - 12s 222ms/step - loss: 0.0538 - accuracy: 0.9797
Epoch 11/100
54/54 [==============================] - 14s 256ms/step - loss: 0.0535 - accuracy: 0.9793
Epoch 12/100
54/54 [==============================] - 11s 211ms/step - loss: 0.0508 - accuracy: 0.9809
Epoch 13/100
54/54 [==============================] - 12s 222ms/step - loss: 0.0496 - accur

54/54 [==============================] - 11s 205ms/step - loss: 0.0408 - accuracy: 0.9858
Epoch 84/100
54/54 [==============================] - 11s 208ms/step - loss: 0.0401 - accuracy: 0.9853
Epoch 85/100
54/54 [==============================] - 12s 231ms/step - loss: 0.0424 - accuracy: 0.9847
Epoch 86/100
54/54 [==============================] - 12s 213ms/step - loss: 0.0492 - accuracy: 0.9816
Epoch 87/100
54/54 [==============================] - 12s 231ms/step - loss: 0.0455 - accuracy: 0.9826
Epoch 88/100
54/54 [==============================] - 12s 226ms/step - loss: 0.0432 - accuracy: 0.9848
Epoch 89/100
54/54 [==============================] - 13s 243ms/step - loss: 0.0468 - accuracy: 0.9835
Epoch 90/100
54/54 [==============================] - 13s 236ms/step - loss: 0.0388 - accuracy: 0.9852
Epoch 91/100
54/54 [==============================] - 11s 210ms/step - loss: 0.0399 - accuracy: 0.9848
Epoch 92/100
54/54 [==============================] - 12s 227ms/step - loss: 0.0461 - 

54/54 [==============================] - 12s 218ms/step - loss: 0.0423 - accuracy: 0.9844
Epoch 62/100
54/54 [==============================] - 12s 222ms/step - loss: 0.0394 - accuracy: 0.9843
Epoch 63/100
54/54 [==============================] - 11s 208ms/step - loss: 0.0379 - accuracy: 0.9856
Epoch 64/100
54/54 [==============================] - 13s 242ms/step - loss: 0.0382 - accuracy: 0.9853
Epoch 65/100
54/54 [==============================] - 13s 235ms/step - loss: 0.0438 - accuracy: 0.9834
Epoch 66/100
54/54 [==============================] - 14s 252ms/step - loss: 0.0388 - accuracy: 0.9867
Epoch 67/100
54/54 [==============================] - 11s 213ms/step - loss: 0.0407 - accuracy: 0.9863
Epoch 68/100
54/54 [==============================] - 13s 237ms/step - loss: 0.0420 - accuracy: 0.9835
Epoch 69/100
54/54 [==============================] - 11s 211ms/step - loss: 0.0387 - accuracy: 0.9857
Epoch 70/100
54/54 [==============================] - 13s 246ms/step - loss: 0.0364 - 

54/54 [==============================] - 13s 244ms/step - loss: 0.0367 - accuracy: 0.9862
Epoch 40/100
54/54 [==============================] - 12s 222ms/step - loss: 0.0356 - accuracy: 0.9873
Epoch 41/100
54/54 [==============================] - 14s 257ms/step - loss: 0.0365 - accuracy: 0.9858
Epoch 42/100
54/54 [==============================] - 14s 261ms/step - loss: 0.0317 - accuracy: 0.9885
Epoch 43/100
54/54 [==============================] - 13s 239ms/step - loss: 0.0338 - accuracy: 0.9879
Epoch 44/100
54/54 [==============================] - 14s 267ms/step - loss: 0.0409 - accuracy: 0.9842
Epoch 45/100
54/54 [==============================] - 14s 254ms/step - loss: 0.0380 - accuracy: 0.9861
Epoch 46/100
54/54 [==============================] - 13s 246ms/step - loss: 0.0359 - accuracy: 0.9871
Epoch 47/100
54/54 [==============================] - 12s 223ms/step - loss: 0.0324 - accuracy: 0.9887
Epoch 48/100
54/54 [==============================] - 12s 226ms/step - loss: 0.0372 - 

In [9]:
np.average(accuracies)

0.8886039710898512

In [10]:
test_generator = datagen.flow_from_directory(
    test_dir_name,
    target_size=img_size,
    color_mode='rgba',
    batch_size=batch_size,
    class_mode='categorical')

Found 1699 images belonging to 2 classes.


In [11]:
loss, acc = model.evaluate_generator(test_generator)
acc

0.7192466259002686